# Stage 1 - Generate a test dataset

## Step 1 - Choose 5 cities

According to the website of SMOOVE, before the year of 2020, SMOOVE had already run bike sharing service in 8 cities in France. In this project, we won't care about electric bikes and the long-term rental bikes. We only focus on the classical bikes in self-service (Vélos en libre-service). For convenience, we call them "bikes" in the rest of this project. We can get the number of bikes and stations from the website. 

We need to pick 5 lucky cities out of these 8. The number of bikes in Paris is so large that it could break the balence of the dataset; there are only 50 electric bikes in Vannes; in Strasbourg, there are so much more bikes in long-term rent (9000)than classical bikes (200).

Finally the 5 cities are : Montpellier, Clermont-Ferrand, Saint-Etienne, Avignon, Belfort.

## Step 2 - Define the structure of this dataset

5*366=1830 Rows: One row per day and per city.

9 Columns: Date, City, Number of available bikes, Number of stations, Number of users who have used bikes on the day, Number of trips, Total travelled distance(km), Total travelled time(hours), Number of new users.

So the shape of this dataset will be (1830, 9)

## Step 3 - Generate the dataset

### Step 3.1 - Date

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Create a list containing all dates of 2020

dates = [datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start = '2020-01-01', end = '2020-12-31'))]

In [3]:
# Create a DataFrame repeating each date 5 times

dataset = pd.DataFrame(np.repeat(dates, 5, axis = 0), columns = ['Date'])

In [4]:
dataset.head(10)

,Date
0,2020-01-01
1,2020-01-01
2,2020-01-01
3,2020-01-01
4,2020-01-01
5,2020-01-02
6,2020-01-02
7,2020-01-02
8,2020-01-02
9,2020-01-02


In [5]:
dataset.shape

(1830, 1)

### Step 3.2 - City

In [6]:
# List of 5 cities

cities = ['Montpellier', 'Clermont-Ferrand', 'Saint Etienne', 'Avignon', 'Belfort']

In [7]:
# Add City column, 5 cities appear in a same order in all the 366 days

dataset.insert(dataset.shape[1], 'City', np.tile(cities, 366))

In [8]:
dataset.head(10)

,Date,City
0,2020-01-01,Montpellier
1,2020-01-01,Clermont-Ferrand
2,2020-01-01,Saint Etienne
3,2020-01-01,Avignon
4,2020-01-01,Belfort
5,2020-01-02,Montpellier
6,2020-01-02,Clermont-Ferrand
7,2020-01-02,Saint Etienne
8,2020-01-02,Avignon
9,2020-01-02,Belfort


In [9]:
dataset.shape

(1830, 2)

### Step 3.3 - Number of available bikes

In this case I operate a fleet of 1000 bikes over 5 different cities. According to the bikes amounts shown by the website of SMOOVE, we roughly estimated the original amounts of bikes in our case using equal scale scaling. 

In [10]:
# Amounts from website

amounts_SMOOVE = [548, 632, 364, 300, 250]

In [11]:
# Estimated amounts

s_ = sum(amounts_SMOOVE)
amounts = [round(x/s_*1000) for x in amounts_SMOOVE]

In [12]:
# Check the sum of amounts equals to 1000 or not

print("The total amount of bikes in 5 cities is", sum(amounts))

The total amount of bikes in 5 cities is 1000


In [13]:
# Print bikes amount in each city

bike_city = pd.DataFrame({'City': pd.Series(cities), 'Total_bike_amount': pd.Series(amounts)})

As we know sometimes some bikes are not available, for example it could be broken, stolen, got out of service region etc. But we have experienced teams to handle these kind of problems timely to make sure there will be at most 5% unavailable bikes every day.

In [14]:
# Add the limit of unavailable bikes for each city

bike_city.insert(bike_city.shape[1], 'Max_unavailable_bike', [round(x*0.05) for x in amounts])

In [15]:
# Take a look at of the results of bike amounts

bike_city

,City,Total_bike_amount,Max_unavailable_bike
0,Montpellier,262,13
1,Clermont-Ferrand,302,15
2,Saint Etienne,174,9
3,Avignon,143,7
4,Belfort,119,6


We assume that for each city and for each single day, the number of unavailable bikes is a random variable who follows a uniform distribution in the set of integers from 0 up to its max unavailable bikes.

Now we can add the column : Number of available bikes into our dataset based on our assumptions.

In [16]:
# Repeat bike_city 366 times for 366 days.

bike_city_new = bike_city
for i in range(365):
    bike_city_new = pd.concat([bike_city_new, bike_city])

In [17]:
bike_city_new.index = dataset.index

In [18]:
# Calculate  "number of available bikes" for each city and for each day

Nb_available_bike = bike_city_new.Total_bike_amount - np.random.randint(0,bike_city_new.Max_unavailable_bike+1)

In [19]:
# Add this column into our dataset

dataset.insert(dataset.shape[1], 'Nb_available_bike', Nb_available_bike)

In [20]:
dataset.head(10)

,Date,City,Nb_available_bike
0,2020-01-01,Montpellier,255
1,2020-01-01,Clermont-Ferrand,295
2,2020-01-01,Saint Etienne,167
3,2020-01-01,Avignon,142
4,2020-01-01,Belfort,118
5,2020-01-02,Montpellier,258
6,2020-01-02,Clermont-Ferrand,300
7,2020-01-02,Saint Etienne,169
8,2020-01-02,Avignon,142
9,2020-01-02,Belfort,119


In [21]:
dataset.shape

(1830, 3)

### Step 3.4 - Number of stations

After observing the data from SMOOVE, I found that the number of stations are roughly equal to or slightly larger than 10% of bike amounts. 

So I decided to set 110 stations for these 1000 bikes. I also used equal scale scaling to arrange the stations into 5 cities, respectly.

In [22]:
# Data from SMOOVE

stations_SMOOVE = [54, 52, 38, 30, 29]

In [23]:
# Data we will use

s_1 = sum(stations_SMOOVE)
stations = [round(x/s_1*110) for x in stations_SMOOVE]

In [24]:
# Check the sum of stations equals to 1000 or not

print("The total amount of stations in 5 cities is", sum(stations))

The total amount of stations in 5 cities is 110


In [25]:
# Print stations amount in each city

station_city = pd.DataFrame({'City': pd.Series(cities), 'Station_amount': pd.Series(stations)})

In [26]:
# Take a look at of the results of station amounts

station_city

,City,Station_amount
0,Montpellier,29
1,Clermont-Ferrand,28
2,Saint Etienne,21
3,Avignon,16
4,Belfort,16


We assume that the number of stations in these 5 cities are invariable in 2020.

In [27]:
# Add "Nb_station" column into our dataset

dataset.insert(dataset.shape[1], 'Nb_station', np.tile(stations, 366))

In [28]:
dataset.head(10)

,Date,City,Nb_available_bike,Nb_station
0,2020-01-01,Montpellier,255,29
1,2020-01-01,Clermont-Ferrand,295,28
2,2020-01-01,Saint Etienne,167,21
3,2020-01-01,Avignon,142,16
4,2020-01-01,Belfort,118,16
5,2020-01-02,Montpellier,258,29
6,2020-01-02,Clermont-Ferrand,300,28
7,2020-01-02,Saint Etienne,169,21
8,2020-01-02,Avignon,142,16
9,2020-01-02,Belfort,119,16


In [29]:
dataset.shape

(1830, 4)

### Step 3.5 - Number of users who have used bikes on the day

Here comes the most important part. The following 3 columns will need the data in this column.

I proposed a mathematical model to estimate the number of users in a single day in which I took 5 factors into account : The population, the congestion level, the density of stations, people's enthusiasm for fitness through cycling and the popularity level of our bikes.

This model is reasonable from a marketing point of view. 

A behavior of consumption will occur only when these 4 steps occur : One has a demand to ride a bike --> he chooses our bikes --> he arrives at one of our stations --> there exist available bikes.

Or : One has a demand to ride a bike --> he arrives at one of ours stations by coincidence --> he thinks our bikes are acceptable --> there exist available bikes.

The orders' exchange of the 2nd and the 3rd step doesn't matter for the model thanks to commutative law of multiplication.

I assumed that the demand of a user to ride a bike comes from 2 parts : the transportation requirements using a bike thanks to congestion level and the fitness requirements which is related to air quality. The popularity level represents the possibility of users to choose our bikes or accept our bikes in the case that he arrives at the station by coincidence. The density of stations represents the difficulty level for users to get our service or the probability for a user to meet one of our station by coincidence. Finaly, we use the average available bike amount per station to evaluate this probability.

 Then apply multiplication principle and addtion principle, we conclude that :

###### Number of users = r * Population

###### r = (a_1 * Congestion level + a_2 * Fitness enthusiasm level) * a_3 * Popularity level * a_4 * Density of stations * a_5 * Bike per station + error

a_1, a_2, a_3, a_4 and a_5 are parameters we need to choose. As to the 5 independent variables, we can estimate them with the help of some data from Internet.

Although this model may not be able to reflet all aspects in real cases, for example the influence of climate, seasons and Covid-19, but it's reliable enough to generate the dataset. And it's easy to modify if we need a new factor.

#### Step 3.5.1 - Population

Through the data on the website of "insee", we can get the total population of the 5 cities in 2019. We regard them as the population we need in 2020 and ignore the daily changes.

In [30]:
population = [298933, 150596, 175792, 92821, 47242]

#### Step 3.5.2 - Congestion level

Through the data of traffic index on the website of "tomtom", we can get the congestion level of 5 cities in 2021 and their changes from 2020. To get the results in 2020, we need a simple calculation. We assume that for citizens, the worse the traffic is, the more possible to transport via bike. 

In [31]:
congestions_2021 = pd.DataFrame({'2021':[0.27, 0.22, 0.19, 0.19, 0.1], 'Change_from_2020':[0.03, 0.01, 0.02, 0.02, 0]})

In [32]:
congestions = congestions_2021['2021'] - congestions_2021.Change_from_2020

In [33]:
congestions

0    0.24
1    0.21
2    0.17
3    0.17
4    0.10
dtype: float64

#### Step 3.5.3 - Fitness enthusiasm level

We assume that for citizens, the better the air quality is, the more possible to do outside exercises. I recorded a list of air pollution level data from "aqicn". Assume that ftiness enthusiasm level = 1/air pollution level.

In [34]:
aqi = [40, 37, 46, 40, 26]

#### Step 3.5.4 - Density of stations

I got the urban area size from Wikipedia. (Superfice en km2)

In [35]:
urban_size = [56.88, 42.67, 79.97, 64.78, 17.1]

Now we merge the results in the step into a DataFrame.

In [36]:
# Create a DataFrame

factors = station_city.copy()

# Add the results one by one

factors.insert(factors.shape[1], 'Population', population)
factors.insert(factors.shape[1], 'Congestion_level', congestions)
factors.insert(factors.shape[1], 'Air_pollution_level', aqi)
factors.insert(factors.shape[1], 'Urban_size', urban_size)

# Calculate the denstities of staions

factors['Station_density'] = factors.Station_amount/factors.Urban_size

In [37]:
# Take a look of it.

factors

,City,Station_amount,Population,Congestion_level,Air_pollution_level,Urban_size,Station_density
0,Montpellier,29,298933,0.24,40,56.88,0.509845
1,Clermont-Ferrand,28,150596,0.21,37,42.67,0.656199
2,Saint Etienne,21,175792,0.17,46,79.97,0.262598
3,Avignon,16,92821,0.17,40,64.78,0.246990
4,Belfort,16,47242,0.10,26,17.10,0.935673


#### Step 3.5.5 - Popularity level

I trust the company was getting better and better day by day, but it's silly to let popularity be monotone increasing. Instead, for every sigle day, I will pick a random value which follows a normal distribution. And the expectation of the distribution is monotone increasing, the variances are the same.

In [38]:
popularity = []

for i in range(366):
    a = i*0.01 + 10
    x = np.random.normal(a,0.5)
    popularity.append(x)

In [39]:
# Look at the popularity I've created

popularity

[10.810616247337931,
 10.386242776305586,
 9.607733218066901,
 9.159503735349865,
 9.852001907459751,
 10.391256360464029,
 10.262114623696872,
 9.790618484890969,
 9.656240806849405,
 10.171949141631005,
 9.74886736786236,
 9.802926164453039,
 10.530332098351344,
 10.325290744025667,
 10.346898453929038,
 11.154681887587522,
 10.28297754975209,
 9.474557207432024,
 9.75792969395666,
 10.600666223296862,
 10.748119095657064,
 10.633788487991888,
 10.326723163559857,
 10.52329253242365,
 9.257462332137537,
 10.27809224764148,
 10.902744657547942,
 10.659109657931172,
 10.818545796549174,
 10.092988200409842,
 11.014279072019622,
 9.896517653402212,
 9.775313321724317,
 10.501745856269112,
 9.86946565486218,
 9.938913266458322,
 10.630949227188491,
 10.200898054836596,
 10.581600927542262,
 10.75183471663667,
 10.854819801468055,
 9.482197480462812,
 10.70215340084945,
 10.2947192828366,
 10.103578785356104,
 9.842102595218101,
 10.87692539674312,
 9.825204064968801,
 10.47677896139208,


In [40]:
factors_final = factors

for i in range(365):
    factors_final = pd.concat([factors_final, factors])

In [41]:
factors_final.insert(factors_final.shape[1], 'Popularity_level', np.repeat(popularity, 5))
factors_final.index = dataset.index

#### Step 3.5.6 - Bike per station

In [42]:
bike_per_station = dataset.Nb_available_bike / dataset.Nb_station

In [43]:
factors_final.insert(factors.shape[1], 'Bike_per_station', bike_per_station)

In [44]:
factors_final.head(10)

,City,Station_amount,Population,Congestion_level,Air_pollution_level,Urban_size,Station_density,Bike_per_station,Popularity_level
0,Montpellier,29,298933,0.24,40,56.88,0.509845,8.793103,10.810616
1,Clermont-Ferrand,28,150596,0.21,37,42.67,0.656199,10.535714,10.810616
2,Saint Etienne,21,175792,0.17,46,79.97,0.262598,7.952381,10.810616
3,Avignon,16,92821,0.17,40,64.78,0.246990,8.875000,10.810616
4,Belfort,16,47242,0.10,26,17.10,0.935673,7.375000,10.810616
5,Montpellier,29,298933,0.24,40,56.88,0.509845,8.896552,10.386243
6,Clermont-Ferrand,28,150596,0.21,37,42.67,0.656199,10.714286,10.386243
7,Saint Etienne,21,175792,0.17,46,79.97,0.262598,8.047619,10.386243
8,Avignon,16,92821,0.17,40,64.78,0.246990,8.875000,10.386243
9,Belfort,16,47242,0.10,26,17.10,0.935673,7.437500,10.386243


In [45]:
factors_final.tail(10)

,City,Station_amount,Population,Congestion_level,Air_pollution_level,Urban_size,Station_density,Bike_per_station,Popularity_level
1820,Montpellier,29,298933,0.24,40,56.88,0.509845,8.793103,13.465953
1821,Clermont-Ferrand,28,150596,0.21,37,42.67,0.656199,10.642857,13.465953
1822,Saint Etienne,21,175792,0.17,46,79.97,0.262598,8.285714,13.465953
1823,Avignon,16,92821,0.17,40,64.78,0.246990,8.937500,13.465953
1824,Belfort,16,47242,0.10,26,17.10,0.935673,7.437500,13.465953
1825,Montpellier,29,298933,0.24,40,56.88,0.509845,8.931034,12.612804
1826,Clermont-Ferrand,28,150596,0.21,37,42.67,0.656199,10.392857,12.612804
1827,Saint Etienne,21,175792,0.17,46,79.97,0.262598,7.952381,12.612804
1828,Avignon,16,92821,0.17,40,64.78,0.246990,8.500000,12.612804
1829,Belfort,16,47242,0.10,26,17.10,0.935673,7.062500,12.612804


The DataFrame Factors_final contains everything we need to estimated the daily user numbers, except 5 parameters.

In [46]:
# Define a function to represent the mathematical model to estimate the number of users

def Nb_users(a_1, a_2, a_3, a_4, a_5):
    r = (a_1*factors_final.Congestion_level + a_2/factors_final.Air_pollution_level) * a_3*factors_final.Station_density * a_4*factors_final.Popularity_level * a_5*factors_final.Bike_per_station
    return round(factors_final.Population * r).astype(int)

After a series of tests I set these parameters to [0.1, 0.5, 0.5, 0.04, 0.05]

In [47]:
dataset.insert(dataset.shape[1], 'Nb_users', Nb_users(0.1, 0.5, 0.5, 0.04, 0.05))

In [48]:
dataset.head(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users
0,2020-01-01,Montpellier,255,29,529
1,2020-01-01,Clermont-Ferrand,295,28,388
2,2020-01-01,Saint Etienne,167,21,111
3,2020-01-01,Avignon,142,16,65
4,2020-01-01,Belfort,118,16,103
5,2020-01-02,Montpellier,258,29,514
6,2020-01-02,Clermont-Ferrand,300,28,380
7,2020-01-02,Saint Etienne,169,21,108
8,2020-01-02,Avignon,142,16,62
9,2020-01-02,Belfort,119,16,100


In [49]:
dataset.tail(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users
1820,2020-12-30,Montpellier,255,29,659
1821,2020-12-30,Clermont-Ferrand,298,28,489
1822,2020-12-30,Saint Etienne,174,21,144
1823,2020-12-30,Avignon,143,16,81
1824,2020-12-30,Belfort,119,16,129
1825,2020-12-31,Montpellier,259,29,627
1826,2020-12-31,Clermont-Ferrand,291,28,447
1827,2020-12-31,Saint Etienne,167,21,129
1828,2020-12-31,Avignon,136,16,73
1829,2020-12-31,Belfort,113,16,115


In [50]:
dataset.shape

(1830, 5)

### Step 3.6 - Number of trips

As we know one user can finish at least 1 trip. But he can also ride for more than once. This number is a integer >= 1. As this number goes larger, the probability will go down. This remind me of Geometric distribution. So I used it to estimate Number of trips based on Number of users. 

In [51]:
# Define a function to transform number of users to number of trips. 
# Assume that the number of trip for each user is a random variable following a geometric distribustion with p=0.5

def user_to_trip(x):
    l = np.random.geometric(0.5, x)
    return sum(l)

In [52]:
# Apply this function to our dataset then insert it into dataset

nb_trips = dataset.Nb_users.apply(user_to_trip)
dataset.insert(dataset.shape[1], 'Nb_trips', nb_trips)

In [53]:
dataset.head(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips
0,2020-01-01,Montpellier,255,29,529,1072
1,2020-01-01,Clermont-Ferrand,295,28,388,748
2,2020-01-01,Saint Etienne,167,21,111,246
3,2020-01-01,Avignon,142,16,65,113
4,2020-01-01,Belfort,118,16,103,232
5,2020-01-02,Montpellier,258,29,514,1029
6,2020-01-02,Clermont-Ferrand,300,28,380,788
7,2020-01-02,Saint Etienne,169,21,108,221
8,2020-01-02,Avignon,142,16,62,121
9,2020-01-02,Belfort,119,16,100,200


In [54]:
dataset.tail(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips
1820,2020-12-30,Montpellier,255,29,659,1340
1821,2020-12-30,Clermont-Ferrand,298,28,489,982
1822,2020-12-30,Saint Etienne,174,21,144,271
1823,2020-12-30,Avignon,143,16,81,164
1824,2020-12-30,Belfort,119,16,129,249
1825,2020-12-31,Montpellier,259,29,627,1216
1826,2020-12-31,Clermont-Ferrand,291,28,447,853
1827,2020-12-31,Saint Etienne,167,21,129,251
1828,2020-12-31,Avignon,136,16,73,150
1829,2020-12-31,Belfort,113,16,115,234


In [55]:
dataset.shape

(1830, 6)

### Step 3.7 - Total travelled distance

As the name of our company, we want our users to arrive their destinations in 15 minutes. In general, the average speed for a cycler is in the range of 16km/h to 20km/h. That means 4km to 5km per trip. For the users who use our bike as a transportation tool, this distance is more likely to be lower than 4km or 5km. But the users who use our bikes for exercise will ride longer than that. So I pick 4km as the mean value, generate a random distance based on normal distrubution for each trip.

In [56]:
# Define a function to transform number of trips to total travelled distance. 
# Assume that the distance of each trip is a random variable following a normal distribustion with mean value = 4.

def trip_to_distance(x):
    l = np.random.normal(4, 1, x)
    return round(sum(l))

In [57]:
# Add Total_distance_km into our dataset

total_distance = dataset.Nb_trips.apply(trip_to_distance)
dataset.insert(dataset.shape[1], 'Total_distance_km', total_distance)

In [58]:
dataset.head(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km
0,2020-01-01,Montpellier,255,29,529,1072,4302
1,2020-01-01,Clermont-Ferrand,295,28,388,748,3003
2,2020-01-01,Saint Etienne,167,21,111,246,983
3,2020-01-01,Avignon,142,16,65,113,451
4,2020-01-01,Belfort,118,16,103,232,918
5,2020-01-02,Montpellier,258,29,514,1029,4106
6,2020-01-02,Clermont-Ferrand,300,28,380,788,3102
7,2020-01-02,Saint Etienne,169,21,108,221,876
8,2020-01-02,Avignon,142,16,62,121,477
9,2020-01-02,Belfort,119,16,100,200,796


In [59]:
dataset.tail(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km
1820,2020-12-30,Montpellier,255,29,659,1340,5371
1821,2020-12-30,Clermont-Ferrand,298,28,489,982,3943
1822,2020-12-30,Saint Etienne,174,21,144,271,1095
1823,2020-12-30,Avignon,143,16,81,164,643
1824,2020-12-30,Belfort,119,16,129,249,967
1825,2020-12-31,Montpellier,259,29,627,1216,4893
1826,2020-12-31,Clermont-Ferrand,291,28,447,853,3453
1827,2020-12-31,Saint Etienne,167,21,129,251,983
1828,2020-12-31,Avignon,136,16,73,150,602
1829,2020-12-31,Belfort,113,16,115,234,934


In [60]:
dataset.shape

(1830, 7)

### Step 3.8 - Total travelled time

Similarly set mean value at 0.25.

In [61]:
# Define a function to transform number of trips to total travelled time. 
# Assume that the distance of each trip is a random variable following a normal distribustion with mean value = 0.25.

def trip_to_time(x):
    l = np.random.normal(0.25, 0.25, x)
    return round(sum(l))

In [62]:
# Add Total_time_hour into our dataset

total_time = dataset.Nb_trips.apply(trip_to_time)
dataset.insert(dataset.shape[1], 'Total_time_hour', total_time)

In [63]:
dataset.head(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km,Total_time_hour
0,2020-01-01,Montpellier,255,29,529,1072,4302,257
1,2020-01-01,Clermont-Ferrand,295,28,388,748,3003,185
2,2020-01-01,Saint Etienne,167,21,111,246,983,56
3,2020-01-01,Avignon,142,16,65,113,451,32
4,2020-01-01,Belfort,118,16,103,232,918,54
5,2020-01-02,Montpellier,258,29,514,1029,4106,255
6,2020-01-02,Clermont-Ferrand,300,28,380,788,3102,202
7,2020-01-02,Saint Etienne,169,21,108,221,876,59
8,2020-01-02,Avignon,142,16,62,121,477,25
9,2020-01-02,Belfort,119,16,100,200,796,51


In [64]:
dataset.tail(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km,Total_time_hour
1820,2020-12-30,Montpellier,255,29,659,1340,5371,339
1821,2020-12-30,Clermont-Ferrand,298,28,489,982,3943,230
1822,2020-12-30,Saint Etienne,174,21,144,271,1095,66
1823,2020-12-30,Avignon,143,16,81,164,643,48
1824,2020-12-30,Belfort,119,16,129,249,967,56
1825,2020-12-31,Montpellier,259,29,627,1216,4893,301
1826,2020-12-31,Clermont-Ferrand,291,28,447,853,3453,223
1827,2020-12-31,Saint Etienne,167,21,129,251,983,56
1828,2020-12-31,Avignon,136,16,73,150,602,39
1829,2020-12-31,Belfort,113,16,115,234,934,57


In [65]:
dataset.shape

(1830, 8)

### Step 3.9 - Number of new users

The number of new users will related to the population and the popularity.

In [66]:
# Define the ratio of population to turn into new users. It's depends on the change of popularity level.

new_user_ratio = (pd.Series(popularity).pct_change()+0.5)/2500

# Calculate the mean value of the number of new users

factors_final.insert(factors_final.shape[1], 'New_user_ratio', np.repeat(new_user_ratio.to_list(), 5))
factors_final['New_user'] = factors_final.Population*factors_final.New_user_ratio

In [67]:
# Again I will use random.normal to make this number more random around its mean value.

nb_new_user = factors_final.New_user.fillna(0).map(lambda x: round(np.random.normal(x, 3)))

In [68]:
# Add this column into our dataset and set the data of the first day to 0.

dataset.insert(dataset.shape[1], 'Nb_new_users', nb_new_user)
dataset.iloc[:5,8] = 0

# Now the generation of the dataset is done. Take a look of it then write it in a csv file.

In [69]:
dataset.describe()

,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km,Total_time_hour,Nb_new_users
count,1830.000000,1830.000000,1830.000000,1830.000000,1830.000000,1830.000000,1830.000000
mean,194.938251,22.000000,261.407650,521.850820,2087.800000,130.493989,30.741530
std,68.675360,5.622924,205.758725,411.152105,1644.517683,103.154577,17.997217
min,113.000000,16.000000,54.000000,91.000000,367.000000,19.000000,0.000000
25%,138.000000,16.000000,101.000000,200.000000,798.250000,49.000000,16.000000
50%,170.000000,21.000000,126.000000,255.000000,1024.500000,64.000000,29.000000
75%,259.000000,28.000000,457.000000,912.750000,3649.000000,229.000000,39.000000
max,302.000000,29.000000,712.000000,1475.000000,5874.000000,390.000000,84.000000


In [70]:
dataset.shape

(1830, 9)

In [71]:
dataset.head(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km,Total_time_hour,Nb_new_users
0,2020-01-01,Montpellier,255,29,529,1072,4302,257,0
1,2020-01-01,Clermont-Ferrand,295,28,388,748,3003,185,0
2,2020-01-01,Saint Etienne,167,21,111,246,983,56,0
3,2020-01-01,Avignon,142,16,65,113,451,32,0
4,2020-01-01,Belfort,118,16,103,232,918,54,0
5,2020-01-02,Montpellier,258,29,514,1029,4106,255,60
6,2020-01-02,Clermont-Ferrand,300,28,380,788,3102,202,28
7,2020-01-02,Saint Etienne,169,21,108,221,876,59,35
8,2020-01-02,Avignon,142,16,62,121,477,25,23
9,2020-01-02,Belfort,119,16,100,200,796,51,5


In [72]:
dataset.tail(10)

,Date,City,Nb_available_bike,Nb_station,Nb_users,Nb_trips,Total_distance_km,Total_time_hour,Nb_new_users
1820,2020-12-30,Montpellier,255,29,659,1340,5371,339,63
1821,2020-12-30,Clermont-Ferrand,298,28,489,982,3943,230,27
1822,2020-12-30,Saint Etienne,174,21,144,271,1095,66,38
1823,2020-12-30,Avignon,143,16,81,164,643,48,16
1824,2020-12-30,Belfort,119,16,129,249,967,56,5
1825,2020-12-31,Montpellier,259,29,627,1216,4893,301,50
1826,2020-12-31,Clermont-Ferrand,291,28,447,853,3453,223,28
1827,2020-12-31,Saint Etienne,167,21,129,251,983,56,31
1828,2020-12-31,Avignon,136,16,73,150,602,39,20
1829,2020-12-31,Belfort,113,16,115,234,934,57,5


In [73]:
dataset.to_csv('test_dataset.csv', index = False, header = True)